https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html

前処理大全だとp.39


In [1]:
import pandas as pd
import numpy as np
pd.options.display.notebook_repr_html = False  # jupyter notebook上での出力形式を制御するために書いています。無くても動きます。

In [2]:
# 動作環境の確認
print(pd.__version__)
print(np.__version__)

1.1.2
1.19.1


## まず、公式ページにも載っている例。


In [3]:
df = pd.DataFrame({'A': range(1, 6),
                   'B': range(10, 0, -2),
                   'C C': range(10, 5, -1)})
df

   A   B  C C
0  1  10   10
1  2   8    9
2  3   6    8
3  4   4    7
4  5   2    6

In [4]:
# 正しい
df.query('A > B')

   A  B  C C
4  5  2    6

In [5]:
# 間違えて文字列じゃない状態で指定した
df.query(A > B)

NameError: name 'A' is not defined

In [6]:
# 間違えて、文字列ではないものを指定した
df.query(['this', 'is', 'list'])

ValueError: expr must be a string to be evaluated, <class 'list'> given

0.25 以降で、列名にスペースが入っていても`列名` をつかって動くようになった。
1.0以降で、

https://stackoverflow.com/questions/50697536/pandas-query-function-not-working-with-spaces-in-column-names



In [7]:
# 正しい
# 列名にスペースが入っている
df.query('B == `C C`')

   A   B  C C
0  1  10   10

In [8]:
# 列名にスペースが入っている。backquoteしてない。
df.query('B == C C')

SyntaxError: invalid syntax (<unknown>, line 1)

In [9]:
df.query('B == "C C"')

Empty DataFrame
Columns: [A, B, C C]
Index: []

In [10]:
df.query('"C C"　== B')

SyntaxError: invalid character in identifier (<unknown>, line 1)

---

In [11]:
df.query('A < B and B < `C C`')

   A  B  C C
1  2  8    9
2  3  6    8

In [12]:
df.query('A < B & B < `C C`')

   A  B  C C
1  2  8    9
2  3  6    8

---

In [13]:
df.query('A == 3')

   A  B  C C
2  3  6    8

In [14]:
df.query('A = 3')

ValueError: cannot assign without a target object

In [15]:
df.query('"A" == 3')

TypeError: argument of type 'int' is not iterable

## 右辺（比較対象の値）に変数を使う場合
@で入れるのが正しい。

In [16]:
# 正しい
three = 3
df.query('A == @three')

   A  B  C C
2  3  6    8

In [17]:
three = 3
df.query('A == three')

UndefinedVariableError: name 'three' is not defined

## 左辺（列名）に変数を使う場合
f文字列で入れるのが正しい。

In [18]:
col_name = "A"

In [19]:
# 正しい
df.query(f'{col_name} == 3')

   A  B  C C
2  3  6    8

In [20]:
# 正しい
df.query(col_name + '== 3')

   A  B  C C
2  3  6    8

In [21]:
df.query('"col_name" == 3')

TypeError: argument of type 'int' is not iterable

In [22]:
df.query('@col_name == 3')

TypeError: argument of type 'int' is not iterable

---

文字列は別種類のでquoteしなきゃ。

http://nekoyukimmm.hatenablog.com/entry/2015/11/30/134704

---

列名が数字から始まっているとダメ：

https://stackoverflow.com/questions/27787264/pandas-query-throws-error-when-column-name-starts-with-a-number


In [23]:
data = {'ab': [1,2,3], 'c1': [1,2,3], 'd': [1,2,3], 'e_f': [1,2,3]}
df = pd.DataFrame(data)
for cl in df.columns:
    print(len(df.query('%s==2' %cl)))

1
1
1
1


In [24]:
data = {'ab': [1,2,3], 'c1': [1,2,3], '1d': [1,2,3], 'e_f': [1,2,3]}
df = pd.DataFrame(data)
for cl in df.columns:
    print(len(df.query('%s==2' %cl)))

1
1


SyntaxError: invalid syntax (<unknown>, line 1)

In [25]:
data = {'ab': [1,2,3], 'c1': [1,2,3], '1d': [1,2,3], 'e_f': [1,2,3]}
df = pd.DataFrame(data)
for cl in df.columns:
    print(cl)
    print(type(cl))
    print(df.query('@cl==2' ))

ab
<class 'str'>


TypeError: argument of type 'int' is not iterable

In [26]:
# New in version 1.0.0: Expanding functionality of backtick quoting for more than only spaces.
data = {'ab': [1,2,3], 'c1': [1,2,3], '1d': [1,2,3], 'e_f': [1,2,3]}
df = pd.DataFrame(data)
df.query('`1d`==2' )

   ab  c1  1d  e_f
1   2   2   2    2